## Download buildings

In [1]:
import geopandas as gpd

In [2]:
buildings_url = 'https://inspire-geoportal.lt/resources/atom/bu/data/BU_INSPIRE2.zip'

In [3]:
!wget https://inspire-geoportal.lt/resources/atom/bu/data/BU_INSPIRE2.zip

--2024-11-29 11:55:32--  https://inspire-geoportal.lt/resources/atom/bu/data/BU_INSPIRE2.zip
Resolving inspire-geoportal.lt (inspire-geoportal.lt)... 46.255.211.30
Connecting to inspire-geoportal.lt (inspire-geoportal.lt)|46.255.211.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 342523846 (327M) [application/zip]
Saving to: ‘BU_INSPIRE2.zip’

BU_INSPIRE2.zip     100%[===================>] 326,66M  3,19MB/s    in 81s     

2024-11-29 11:56:54 (4,02 MB/s) - ‘BU_INSPIRE2.zip’ saved [342523846/342523846]



In [3]:
!unzip BU_INSPIRE2.zip

Archive:  BU_INSPIRE2.zip
  inflating: BU_INSPIRE2.gml         


In [4]:
buildings = gpd.read_file('BU_INSPIRE2.gml')

/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: Field with same name (description) already exists in (Building). Skipping newer ones
  return ogr_read(


### Update region dict and save buildings

In [9]:
buildings = buildings.to_crs(epsg=3035)

In [23]:
# add to regions dir

In [10]:
bounds = buildings.bounds

In [11]:
from shapely import box

In [15]:
lithuania_convex_hull = box(bounds.minx.min(), bounds.miny.min(), bounds.maxx.max(), bounds.maxy.max())

In [45]:
regions_datadir = '/data/uscuni-ulce/'
region_hulls = gpd.read_parquet(
        regions_datadir + "regions/" + "cadastre_regions_hull.parquet"
    )

In [46]:
lithuania_region_id = region_hulls.index[-1] + 100

In [47]:
### add lithuania to the region_hulls
region_hulls.loc[lithuania_region_id, 'convex_hull'] = lithuania_convex_hull

In [48]:
region_hulls

,convex_hull
labels,
4,"POLYGON ((4122200 3045600, 4121500 3045900, 40..."
10,"POLYGON ((4068900 3057300, 4051000 3064300, 40..."
132,"POLYGON ((4053100 3049800, 4050900 3053000, 40..."
134,"POLYGON ((4052500 3093600, 4052200 3093700, 40..."
286,"POLYGON ((4060600 3103900, 4057000 3104200, 40..."
...,...
137636,"POLYGON ((5221900 3158300, 5213500 3166400, 52..."
137863,"POLYGON ((5243700 3122100, 5235000 3122700, 52..."
138804,"POLYGON ((5290800 3173900, 5280100 3176400, 52..."


In [49]:
region_hulls.to_parquet(regions_datadir + "regions/" + "cadastre_regions_hull.parquet")

In [50]:
buildings.to_parquet(f'/data/uscuni-ulce/buildings_standardised/buildings_lithuania.pq')

In [ ]:
buildings.to_parquet(f'/data/uscuni-ulce/buildings_in_regions/buildings_{lithuania_region_id}.pq')

## Download Streets

In [63]:
import datetime
streets_dir = '/data/uscuni-ulce/overture_streets/'
from core.generate_streets import read_overture_region_streets
def process_and_save(region_hull, region_id):
    print('Processing', region_id, datetime.datetime.now())
    streets = read_overture_region_streets(region_hull, region_id)
    streets.to_parquet(streets_dir + f'streets_{region_id}.pq')

In [64]:
region_hull = region_hulls.to_crs(epsg=4326).loc[lithuania_region_id].iloc[0]
region_id = lithuania_region_id

In [66]:
%%time
process_and_save(region_hull, region_id)

Processing 139196 2024-11-29 12:27:27.560923
CPU times: user 14.1 s, sys: 5.13 s, total: 19.2 s
Wall time: 5min 9s


In [67]:
gdf = gpd.read_parquet(streets_dir + f'streets_{lithuania_region_id}.pq')

In [69]:
lithuania_region_id

np.int64(139196)